## Setup required for all notebooks
---------------------
*This notebook works best with the conda_python3 kernel on a ml.t3.medium machine*.

**This step of our solution design covers running setup steps that need to be run prior to any other notebook being run.**

1. Prerequisite: a `Python 3.11` conda environment.


#### Import all of the necessary libraries below to run this notebook

In [ ]:
# if interactive mode is set to no -> pickup fmbench from Python installation path
# if interactive mode is set to yes -> pickup fmbench from the current path (one level above this notebook)
# if interactive mode is not defined -> pickup fmbench from the current path (one level above this notebook)
# the premise is that if run non-interactively then it can only be run through main.py which will set interactive mode to no
import os
import sys
if os.environ.get("INTERACTIVE_MODE_SET", "yes") == "yes":
    sys.path.append(os.path.dirname(os.getcwd()))

#### Install packages listed in the requirements.txt file

In [ ]:
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import sys
import time
import json
import boto3
import asyncio
import logging
import importlib.util
import fmbench.scripts
from pathlib import Path
from fmbench.utils import *
from fmbench.globals import *
from typing import Dict, List, Optional
from sagemaker import get_execution_role
import importlib.resources as pkg_resources
from botocore.exceptions import ClientError
from botocore.exceptions import NoCredentialsError

#### Pygmentize globals.py to view and use any of the globally initialized variables 

#### Set up a logger to log all messages while the code runs

In [ ]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Load the config.yml file
------

The config.yml file contains information that is used across this benchmarking environment, such as information about the aws account, prompts, payloads to be used for invocations, and model configurations like the version of the model, the endpoint name, model_id that needs to be deployed. Configurations also support the gives instance type to be used, for example: "ml.g5.24xlarge", the image uri, whether or not to deploy this given model, followed by an inference script "jumpstart.py" which supports the inference script for jumpstart models to deploy the model in this deploy notebook. 

View the contents of the config yml file below and how it is loaded and used throughout this notebook with deploying the model endpoints asynchronously.

In [ ]:
## Load the config.yml file referring to the globals.py file
config = load_config(CONFIG_FILE)

## configure the aws region and execution role
aws_region = config['aws']['region']


try:
    sagemaker_execution_role = get_execution_role()
    config['aws']['sagemaker_execution_role'] = sagemaker_execution_role
    logger.info(f"determined SageMaker exeuction role from get_execution_role")
except Exception as e:
    logger.error(f"could not determine SageMaker execution role, error={e}")
    logger.info(f"going to look for execution role in config file..")
    sagemaker_execution_role = config['aws'].get('sagemaker_execution_role')
    if sagemaker_execution_role is not None:
        logger.info(f"found SageMaker execution role in config file..")

logger.info(f"aws_region={aws_region}, execution_role={config['aws']['sagemaker_execution_role']}")
logger.info(f"config={json.dumps(config, indent=2)}")

#### Download any scripts from Amazon S3

Users can upload scripts to S3 which contain custom code for deployment and inference. We download these scripts here and place them in the `fmbench` package installation directory.

In [ ]:
# Initialize the S3 client
s3_client = boto3.client('s3', region_name=aws_region)

# Assuming fmbench is a valid Python package and scripts is a subdirectory within it
scripts_dir = Path(pkg_resources.files('fmbench'), 'scripts')
logger.info(f"Using fmbench.scripts directory: {scripts_dir}")

# Ensure the scripts directory exists
scripts_dir.mkdir(parents=True, exist_ok=True)

read_bucket = config['s3_read_data']['read_bucket']
logger.info(f"the read bucket is --> {read_bucket} for reading the script files")
scripts_prefix = config['s3_read_data']['scripts_prefix']
logger.info(f"the scripts directory is --> {scripts_prefix} for reading the script file names")
script_files = config['s3_read_data'].get('script_files', [])
if script_files is None:
    script_files = []
logger.info(f"Extracted script files that the user has provided --> {script_files}")

# Download script files to the fmbench.scripts directory
try:
    for script_name in script_files:
        # do os.path.join
        s3_script_path = f"{scripts_prefix}/{script_name}"
        ## take this out of the loop 
        logger.info(f"the script path for where the scripts you have entered in s3 will be installed --> {s3_script_path}")
        local_script_path = scripts_dir / script_name
        logger.info(f"Downloading {s3_script_path} to {local_script_path}")
        txt = read_from_s3(read_bucket, s3_script_path)
        if txt:
            Path(local_script_path).write_text(txt)
        else:
            logger.error(f"nothing read from {s3_script_path}")
        #s3_client.download_file(read_bucket, s3_script_path, str(local_script_path))
except ClientError as error:
    logger.error(f"Failed to download script files: {error}")

#### Download any specified config .yml files from S3

In [ ]:
## Intialize the config dir that contains the config files for fmbench/to download config files from s3 into
config_dir = Path(pkg_resources.files('fmbench'), 'configs')
logger.info(f"Using fmbench.configs directory: {config_dir}")

# Ensure the scripts directory exists
config_dir.mkdir(parents=True, exist_ok=True)
logger.info(f"the read bucket is --> {read_bucket} for reading the config files")
# config prefix for all of the config files
config_prefix = config['s3_read_data']['configs_prefix']
logger.info(f"the configs directory is --> {config_prefix} for reading the config file names")

## all config files that reside within s3 as specified in the config file being used
config_files = config['s3_read_data'].get('config_files', [])
logger.info(f"Extracted config files that the user has provided --> {config_files}")

# Download config files to the fmbench.configs directory
try:
    for config_file in config_files:
        # do os.path.join
        s3_config_file_path = f"{config_prefix}/{config_file}"
        ## take this out of the loop 
        logger.info(f"the config file path for where the config files you have entered in s3 will be installed --> {s3_config_file_path}")
        local_config_path = config_dir / config_file
        logger.info(f"Downloading {s3_config_file_path} to {local_config_path}")
        txt = read_from_s3(read_bucket, s3_config_file_path)
        if txt:
            Path(local_config_path).write_text(txt)
        else:
            logger.error(f"nothing read from {s3_config_file_path}")
        #s3_client.download_file(read_bucket, s3_config_file_path, str(local_config_path))
except ClientError as error:
    logger.error(f"Failed to download config files: {error}")

## Sanity checks on config file

In [ ]:
# check if dataset of interest is being tested in at least one experiment
ds_of_interest = config['metrics']['dataset_of_interest']
found = [True for e in config['experiments'] \
         if any([True for f in e['payload_files'] if ds_of_interest in f])]
if not found:
    error_message = f"dataset {ds_of_interest} not found in any of the experiments, \
                      add it to the payload_files section of at least one experiment in the config file, exiting for now"
    logger.error(error_message)
    ValueError(error_message)
    sys.exit(1)